In [ ]:
# BACKTEST VERSION 2.0 ========================================================
# Apply market indicators and basic commonality analysis
# to detect bottom and trigger the buying time via Telegram
# also perform back-testing record -> Save to Picture
# =============================================================================
# Library for process Datetime
from datetime import datetime, timedelta
import time
import pytz
# Library for POST GET requests
import requests
import urllib.parse
import json
# Library for Data Analysis
# from decimal import Decimal
import pandas as pd
import statistics
import matplotlib
matplotlib.use('Agg')  # Use the Agg backend for non-interactive plotting
import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
import os
# Connect to mariaDB to perform CRUD
import mariadb

# Define the GMT+7 timezone
timezone = pytz.timezone('Asia/Bangkok')
# Set the interval for the status message (in seconds)
interval = 60*60  # 1 hours in seconds
# Check point of important handling - in UNIX timestamp
check_point = time.time()

# Define classification threshold for backtesting
threshold = 0.8

# Database connection parameters
conn_params= {
    "host" : "103.173.66.235",
    # "host" : "localhost",
    "database" : "crypto",
    "user" : "root",
    "password" : "Trien825590"
}
# Define your quiet hours not to send Telegram
quiet_hours = [
    ((22,0),(23,59)),  # 10 PM to midnight
    ((0,0),(6,0))      # Midnight to 6 AM
]

# Function to send a message to Telegram
def send_telegram(message_string, max_retry=2):
    # Telegram bot token and chat ID
    TOKEN = "5614737400:AAHbvZrJbomt09EkpPuhadBCJl7NaGu6rlg"
    ID = "5559031253"

    # Check if the time now is quiet hour or not
    current_time = datetime.now(timezone)
    for start, end in quiet_hours:
        quiet_start = current_time.replace(hour=start[0], minute=start[1], second=0, microsecond=0)
        quiet_end = current_time.replace(hour=end[0], minute=end[1], second=59, microsecond=0)
        if quiet_start <= current_time <= quiet_end:
            return False
    # Then continue process send to Telegram
    current_retry = 0
    while (current_retry <= max_retry):
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        if current_retry > 0:
            print(f"{date_time} Telegram: Retry #{current_retry}")
            time.sleep(5)   # Wait a second to stabilize the server
        try:
            # URL encode the message string
            encoded_message = urllib.parse.quote(message_string)
            # Construct the URL
            url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={ID}&text={encoded_message}"
            # Send the request
            response = requests.post(url, timeout=5)
            response.raise_for_status()
            return True
        except requests.exceptions.RequestException as e:            
            print(date_time, f"ERROR send Telegram: \n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(date_time, "TELEGRAM: Max retries reached.")
                return False
        except requests.exceptions.HTTPError as http_err:
            print(f"Telegram HTTP error occurred: {http_err}")  # Handle the HTTP error
        except Exception as err:
            print(f"Telegram other error occurred: {err}")      # Handle any other errors

# Function get price 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 12h, 1d
def get_data(coin_name, time_unit, time_range, max_retry=5, time_out=10):
    current_retry = 0
    url = "https://api-pro.goonus.io/perpetual/v1/klines"
    # Get the current Unix timestamp value in ms
    current_time = int(time.time()) * 1000
    # Define the parameters for the API request
    params = {
        "symbol": coin_name,
        "interval": time_unit,
        "endTime": current_time,
        # "limit": "600"
    }
    headers = {
        "Content-Type": "application/json",
        "Connection": "keep-alive"
    }
    while current_retry <= max_retry:
        if current_retry > 0:
            print(f"GET Crypto {coin_name}: Retry #{current_retry}")
            time.sleep(5)   # Wait a second to stabilize the server
        try:
            # Try request & get the Response during 5 seconds
            response = requests.get(url, params=params, headers=headers, timeout=time_out)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()

            historical_data = response.json()
            # Map the time_range matching with the total record
            while len(historical_data) < time_range:
                time_range -= 10
            # Check if the data has sufficient 100 records
            if len(historical_data) >= 100:
                # Take the latest "time_range" elements in the list
                historical_data = historical_data[-time_range:]
                # [Time,Open,High,Low,Close,Base qty,Quote qty,Time]
                # Filter to only include 'timestamp' and 'close'
                filtered_data = [[
                    # Convert Unix timestamp -> GMT +7 hours (+25200)
                    datetime.fromtimestamp(int(item[0])/1000, tz=pytz.utc).astimezone(timezone).strftime('%Y-%m-%d %H:%M:%S'),
                    # Close price
                    float(item[4]),
                    # Percentage difference
                    round(100*(float(item[2])-float(item[3]))/float(item[1]),2),
                    float(item[2]), float(item[3])
                ] for item in historical_data]
                # [timestamp, close, %price_diff, Highest, Lowest]
                # print(filtered_data)
                return filtered_data
            else:
                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                message = f"Error get {coin_name} - Not enough data ({len(historical_data)})"
                print(date_time, message)
                send_telegram(message)
                return None
        except requests.exceptions.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(date_time, f"ERROR Request {coin_name} - retry #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: Max retries reached. Raising exception.")
                send_telegram(f"ERROR get: {coin_name} - Max retries\n{e}")
                return None
        except requests.exceptions.HTTPError as http_err:
            print(f"Get crypto HTTP error occurred: {http_err}")  # Handle the HTTP error
        except Exception as err:
            print(f"Get crypto other error occurred: {err}")      # Handle any other errors

# Function to Read empty record in MariaDB database
def read_database(null_data = "yes", max_retry=5):
    current_retry = 0
    while current_retry <= max_retry:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        if current_retry > 0:
            print(f"{date_time} READ DB Retry #{current_retry}")
            time.sleep(5)

        # Create connection & implement cursor
        connection = mariadb.connect(**conn_params)
        cursor = connection.cursor()

        try:            
            # Check the null data
            query = "SELECT * FROM `crypto_backtest` ORDER BY `record_id` ASC"
            if null_data == "yes":
                query = "SELECT * FROM `crypto_backtest` WHERE `avg_next30` IS NULL OR `avg_next30` = '' ORDER BY `record_id` ASC"
            elif null_data == "no":
                query = "SELECT * FROM `crypto_backtest` WHERE `avg_next30` IS NOT NULL OR `avg_next30` != '' ORDER BY `record_id` ASC"
            
            cursor.execute(query)
            rows = cursor.fetchall()
            # Close Connection & Free resources
            cursor.close()
            connection.close()
            return rows
        except Exception as e:
            print(f"{date_time}: READ DB failed\n{e}")
            send_telegram(f"READ DB failed #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: READ DB Max retries reached.")
                send_telegram(f"READ DB: Max retries reached.")
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                return None

# Function to Delete expire record in MariaDB database
def delete_database(coin_name, date_trigger, max_retry=5):
    current_retry = 0
    while current_retry <= max_retry:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        if current_retry > 0:
            print(f"{date_time} DELETE DB Retry #{current_retry}")
            time.sleep(5)

        # Create connection & implement cursor
        connection = mariadb.connect(**conn_params)
        cursor = connection.cursor()

        try:            
            # Check if record exist
            query = "SELECT * FROM `crypto_backtest` WHERE coin_name = ? AND date_trigger = ?"
            data = (coin_name, date_trigger)
            cursor.execute(query,data)
            if len(cursor.fetchall()) > 0:                
                query = "DELETE FROM `crypto_backtest` WHERE coin_name = ? AND date_trigger = ?"
                data = (coin_name, date_trigger)
                cursor.execute(query, data)
                connection.commit()
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                print("\t\tSuccessfully delete record")
                return True
            else:
                print("\t\tCannot find the record to delete")
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                return False

        except Exception as e:
            print(f"{date_time}: DELETE DB failed\n{e}")
            send_telegram(f"DELETE DB failed #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: DELETE DB Max retries reached.")
                send_telegram(f"DELETE DB: Max retries reached.")
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                return False

# Function call API to Update in mySQL database ===============================
def update_database(coin_name, date_trigger,
                    avg_next15, min_next15, max_next15,
                    avg_next30, min_next30, max_next30,
                    avg_next45, min_next45, max_next45,
                    avg15_per, min15_per, max15_per,
                    avg30_per, min30_per, max30_per,
                    avg45_per, min45_per, max45_per,
                    result_15, result_30, result_45, max_retry=5):
   
    current_retry = 0
    while current_retry <= max_retry:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        if current_retry > 0:
            print(f"{date_time} UPDATE DB Retry #{current_retry}")
            time.sleep(5)

        # Create connection & implement cursor
        connection = mariadb.connect(**conn_params)
        cursor = connection.cursor()

        try:            
            # Check if record exist
            query = "SELECT * FROM `crypto_backtest` WHERE coin_name = ? AND date_trigger = ?"
            data = (coin_name, date_trigger)
            cursor.execute(query,data)
            if len(cursor.fetchall()) > 0:
                query = "UPDATE `crypto_backtest` SET `avg_next15` = ?, `min_next15` = ?, `max_next15` = ?, `avg_next30` = ?, `min_next30` = ?, `max_next30` = ?, `avg_next45` = ?, `min_next45` = ?, `max_next45` = ?,\
                        `avg15_per` = ?, `min15_per` = ?, `max15_per` = ?, `avg30_per` = ?, `min30_per` = ?, `max30_per` = ?, `avg45_per` = ?, `min45_per` = ?, `max45_per` = ?,\
                        `result_15` = ?, `result_30` = ?, `result_45` = ? WHERE `crypto_backtest`.`coin_name` = ? AND `crypto_backtest`.`date_trigger` = ?"
                data = (    avg_next15, min_next15, max_next15, avg_next30, min_next30, max_next30, avg_next45, min_next45, max_next45,
                            avg15_per, min15_per, max15_per, avg30_per, min30_per, max30_per, avg45_per, min45_per, max45_per,
                            result_15, result_30, result_45, coin_name, date_trigger)
                cursor.execute(query, data)
                connection.commit()
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                print(f"\t\t{coin_name}: Successfully update record")
                return True
            else:
                print("\t\tCannot find the record to update")
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                return False

        except Exception as e:            
            print(f"{date_time}: UPDATE DB failed\n{e}")
            send_telegram(f"UPDATE DB failed #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: UPDATE DB Max retries reached.")
                send_telegram(f"UPDAET DB: Max retries reached.")
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                return False

# Process Back testing ========================================================
def back_testing():
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    print(f"{date_time}: Start Backtesting")

    pending_data = read_database(null_data='yes')
    # print("Pending Data:", len(pending_data), pending_data)j
    
    if pending_data and len(pending_data) > 0:
        print(f"\t\tRemaining {len(pending_data)} record(s)")
        pending_coins = [(item[1], item[3], float(item[4])) for item in pending_data]
        # print("Pending:", pending_coins)
        for remain in pending_coins:
            # [(coin_name, trigger_date, trigger_value)]
            coin_name = remain[0]
            trigger_date = remain[1]
            trigger_value = remain[2]
            
            # Collect data based on each record
            datacoin_backtest = get_data(coin_name, "1m", 600)
            
            # Check if the trigger_date < oldest_date -> Delete record ========================
            oldest_date = datetime.strptime(datacoin_backtest[0][0],'%Y-%m-%d %H:%M:%S')
            # print(oldest_date, trigger_date)
            if (trigger_date <= oldest_date):
                print(f'\t\t{coin_name}: Late record data -> Delete')
                delete_database(coin_name, trigger_date.strftime('%Y-%m-%d %H:%M:%S'))
                continue    # Skip to next record
            
            # Start Performing back Testing ====================================================
            # Calculate the end time (15 minutes after trigger_datetime)
            end_time15 = trigger_date + timedelta(minutes=15)
            end_time30 = trigger_date + timedelta(minutes=30)
            end_time45 = trigger_date + timedelta(minutes=45)
            # print(end_time15, end_time30, end_time45)

            try:
                # Filter out elements that are larger than the trigger_datetime within 15 minutes
                filtered_list15 = [entry for entry in datacoin_backtest if remain[1] < datetime.strptime(entry[0], '%Y-%m-%d %H:%M:%S') <= end_time15]
                filtered_list30 = [entry for entry in datacoin_backtest if remain[1] < datetime.strptime(entry[0], '%Y-%m-%d %H:%M:%S') <= end_time30]
                filtered_list45 = [entry for entry in datacoin_backtest if remain[1] < datetime.strptime(entry[0], '%Y-%m-%d %H:%M:%S') <= end_time45]
                # print(remain, len(filtered_list45))

                # Wait until enough dataset
                if len(filtered_list45) >= 45:
                    # ['timestamp', 'close', 'percent', 'high', 'low']
                    val15_avg = [entry15[1] for entry15 in filtered_list15]
                    val15_min = [entry15[4] for entry15 in filtered_list15]
                    val15_max = [entry15[3] for entry15 in filtered_list15]
                    val30_avg = [entry30[1] for entry30 in filtered_list30]
                    val30_min = [entry30[4] for entry30 in filtered_list30]
                    val30_max = [entry30[3] for entry30 in filtered_list30]
                    val45_avg = [entry45[1] for entry45 in filtered_list45]
                    val45_min = [entry45[4] for entry45 in filtered_list45]
                    val45_max = [entry45[3] for entry45 in filtered_list45]

                    # Calculate in 15
                    avg_value15 = round((statistics.mean(val15_avg) if val15_avg else None), 3)
                    avg15_per = round(100 * (avg_value15 - remain[2]) / remain[2], 3)
                    min_value15 = min(val15_min) if val15_min else None
                    min15_per = round(100 * (min_value15 - remain[2]) / remain[2], 3)
                    max_value15 = max(val15_max) if val15_max else None
                    max15_per = round(100 * (max_value15 - remain[2]) / remain[2], 3)
                    # Compare threshold and Calculate Result 15:
                    res15_avg = 2 if avg15_per >= threshold else 1 if 0 < avg15_per < threshold else -1 if -threshold < avg15_per < 0 else -2 if avg15_per <= -threshold else 0
                    res15_min = 3 if min15_per >= threshold else 2 if 0 < min15_per < threshold else -2 if -threshold < min15_per < 0 else -3 if min15_per <= -threshold else 0
                    res15_max = 4 if max15_per >= threshold else 2 if 0 < max15_per < threshold else -2 if -threshold < max15_per < 0 else -4 if max15_per <= -threshold else 0
                    result_15 = res15_avg + res15_min + res15_max

                    # Calculate in 30
                    avg_value30 = round((statistics.mean(val30_avg) if val30_avg else None), 3)
                    avg30_per = round(100 * (avg_value30 - remain[2]) / remain[2], 3)
                    min_value30 = min(val30_min) if val30_min else None
                    min30_per = round(100 * (min_value30 - remain[2]) / remain[2], 3)
                    max_value30 = max(val30_max) if val30_max else None
                    max30_per = round(100 * (max_value30 - remain[2]) / remain[2], 3)
                    # Compare threshold and Calculate Result 30
                    res30_avg = 2 if avg30_per >= threshold else 1 if 0 < avg30_per < threshold else -1 if -threshold < avg30_per < 0 else -2 if avg30_per <= -threshold else 0
                    res30_min = 3 if min30_per >= threshold else 2 if 0 < min30_per < threshold else -2 if -threshold < min30_per < 0 else -3 if min30_per <= -threshold else 0
                    res30_max = 4 if max30_per >= threshold else 2 if 0 < max30_per < threshold else -2 if -threshold < max30_per < 0 else -4 if max30_per <= -threshold else 0
                    result_30 = res30_avg + res30_min + res30_max

                    # Calculate in 45
                    avg_value45 = round((statistics.mean(val45_avg) if val45_avg else None), 3)
                    avg45_per = round(100 * (avg_value45 - remain[2]) / remain[2], 3)
                    min_value45 = min(val45_min) if val45_min else None
                    min45_per = round(100 * (min_value45 - remain[2]) / remain[2], 3)
                    max_value45 = max(val45_max) if val45_max else None
                    max45_per = round(100 * (max_value45 - remain[2]) / remain[2], 3)
                    # Compare threshold and Calculate Result 45
                    res45_avg = 2 if avg45_per >= threshold else 1 if 0 < avg45_per < threshold else -1 if -threshold < avg45_per < 0 else -2 if avg45_per <= -threshold else 0
                    res45_min = 3 if min45_per >= threshold else 2 if 0 < min45_per < threshold else -2 if -threshold < min45_per < 0 else -3 if min45_per <= -threshold else 0
                    res45_max = 4 if max45_per >= threshold else 2 if 0 < max45_per < threshold else -2 if -threshold < max45_per < 0 else -4 if max45_per <= -threshold else 0
                    result_45 = res45_avg + res45_min + res45_max

                    # Update Database
                    update_database(coin_name, trigger_date.strftime('%Y-%m-%d %H:%M:%S'), avg_value15, min_value15, max_value15, avg_value30, min_value30, max_value30, avg_value45, min_value45, max_value45,
                                    avg15_per, min15_per, max15_per, avg30_per, min30_per, max30_per, avg45_per, min45_per, max45_per, result_15, result_30, result_45)

                    # Draw graph ======================================
                    # Convert to DataFrame
                    df_graph = pd.DataFrame(datacoin_backtest, columns=['timestamp', 'close', 'percent', 'high', 'low'])
                    # Limit to the latest 250 records
                    df_graph = df_graph.tail(250)
                    # Change the "timestamp" to the datetime format
                    df_graph['timestamp'] = pd.to_datetime(df_graph['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
                    # Plot the visualization
                    plt.figure(figsize=(8, 4))
                    plt.title(f'"{coin_name}" at "{trigger_date}" - ({res15_avg}+{res15_min}+{res15_max})/({res30_avg}+{res30_min}+{res30_max})/({res45_avg}+{res45_min}+{res45_max})')
                    plt.plot(df_graph['timestamp'], df_graph['close'], label='Close Price', color='blue')
                    # plt.xlabel('Time')
                    # plt.ylabel('Price')
                    plt.axvline(x = trigger_date, color = 'red', linestyle = '--', label = 'Specific Time')
                    plt.axvline(x = end_time15, color = 'black', linestyle = ':')
                    plt.axvline(x = end_time30, color = 'black', linestyle = ':')
                    plt.axvline(x = end_time45, color = 'black', linestyle = ':')
                    plt.legend()
                    plt.grid(True)
                    # # Save the plot as a file C:\Users\lebti\OneDrive - Schaeffler Technologies AG & Co. KG\3_Project\Tyen\food_tour\graph\2024-12-08_12-58-12_BIGTIMEVNDC.png
                    file_name = rf"C:\inetpub\wwwroot\crypto\{trigger_date.strftime('%Y-%m-%d_%H-%M-%S')}_{coin_name}.png"
                    # print(file_name)
                    plt.savefig(file_name, dpi=100)
                    # plt.show()
                    plt.clf()
                    plt.close('all')
                    print(f"\t\t{coin_name}: Successfully generate graph")

                    time.sleep(5)
                    # if os.path.exists(file_name):
                    #     os.remove(file_name)

                elif len(filtered_list45) < 45:
                    print(f'\t\t{coin_name}: Not enough record ({len(filtered_list45)})')

            except Exception as e:
                print(f"BACKTEST ERROR: {e}")
                send_telegram(f"BACKTEST ERROR: {e}")
    else:
        print(" ===> ERROR:", pending_data)

# MAIN PROGRAM ================================================================
if __name__ == '__main__':
    # Send log to Telegram
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    message = f"{date_time}: Beta started"
    print(message)
    send_telegram(message)

    try:
        # Create connection & implement cursor
        connection = mariadb.connect(**conn_params)
        cursor = connection.cursor()
        # Close Connection & Free resources
        cursor.close()
        connection.close()

        # Infinite loop to execute
        while True:
            back_testing()

            current_time = time.time()
            if (current_time - check_point) >= interval:
                # date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                check_point = current_time
                # Process Back Testing
                send_telegram("Beta still working")

            time.sleep(600)
    except KeyboardInterrupt:
        print("Server is shutting down...")
    finally:
        # Close Connection & Free resources
        cursor.close()
        connection.close()

In [9]:
# Library for process Datetime
from datetime import datetime, timedelta
import time
import pytz
# Library for POST GET requests
import requests
import urllib.parse
import json
# Library for Data Analysis
# from decimal import Decimal
import pandas as pd
import statistics
import matplotlib
matplotlib.use('Agg')  # Use the Agg backend for non-interactive plotting
import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
import os
import shutil
# Connect to mariaDB to perform CRUD
import mariadb


# Define the GMT+7 timezone
timezone = pytz.timezone('Asia/Bangkok')

# Database connection parameters
conn_params= {
    "host" : "103.173.66.235",
    # "host" : "localhost",
    "database" : "crypto",
    "user" : "root",
    "password" : "Trien825590"
}

# Function to Read empty record in MariaDB database
def read_database(null_data = "yes", max_retry=5):
    current_retry = 0
    while current_retry <= max_retry:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        if current_retry > 0:
            print(f"{date_time} READ DB Retry #{current_retry}")
            time.sleep(5)

        # Create connection & implement cursor
        connection = mariadb.connect(**conn_params)
        cursor = connection.cursor()

        try:            
            # Check the null data
            query = "SELECT * FROM `crypto_backtest` ORDER BY `record_id` ASC"
            if null_data == "yes":
                query = "SELECT * FROM `crypto_backtest` WHERE `avg_next30` IS NULL OR `avg_next30` = '' ORDER BY `record_id` ASC"
            elif null_data == "no":
                query = "SELECT * FROM `crypto_backtest` WHERE `avg_next30` IS NOT NULL OR `avg_next30` != '' ORDER BY `record_id` ASC"
            
            cursor.execute(query)
            rows = cursor.fetchall()
            # Close Connection & Free resources
            cursor.close()
            connection.close()
            return rows
        except Exception as e:
            print(f"{date_time}: READ DB failed\n{e}")
            send_telegram(f"READ DB failed #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: READ DB Max retries reached.")
                send_telegram(f"READ DB: Max retries reached.")
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                return None


data = read_database(null_data='no')
print(len(data))
for i in data:
    # print(i)
    coin_name = i[1]
    date_string = i[3]
    result = int(i[26])
    filename = rf"crypto\{date_string.strftime('%Y-%m-%d_%H-%M-%S')}_{coin_name}.png"
    if os.path.exists(filename):
        if result > 0:
            new_filename = rf"positive\{date_string.strftime('%Y-%m-%d_%H-%M-%S')}_{coin_name}.png"
            shutil.copy(filename, new_filename)
        else:
            new_filename = rf"negative\{date_string.strftime('%Y-%m-%d_%H-%M-%S')}_{coin_name}.png"
            shutil.copy(filename, new_filename)
    else:
        print(filename, "NOT EXITS !")
    # print(filename)

1460
crypto\2024-12-08_01-41-16_DOGEVNDC.png NOT EXITS !
crypto\2024-12-08_01-41-16_DOGEVNDC.png
crypto\2024-12-08_02-28-03_ADAVNDC.png NOT EXITS !
crypto\2024-12-08_02-28-03_ADAVNDC.png
crypto\2024-12-08_02-32-33_UNIVNDC.png NOT EXITS !
crypto\2024-12-08_02-32-33_UNIVNDC.png
crypto\2024-12-08_02-37-12_SUIVNDC.png NOT EXITS !
crypto\2024-12-08_02-37-12_SUIVNDC.png
crypto\2024-12-08_02-39-10_GRASSVNDC.png NOT EXITS !
crypto\2024-12-08_02-39-10_GRASSVNDC.png
crypto\2024-12-08_02-40-07_LYNXVNDC.png NOT EXITS !
crypto\2024-12-08_02-40-07_LYNXVNDC.png
crypto\2024-12-08_02-40-09_WLDVNDC.png NOT EXITS !
crypto\2024-12-08_02-40-09_WLDVNDC.png
crypto\2024-12-08_02-40-18_SOLVNDC.png NOT EXITS !
crypto\2024-12-08_02-40-18_SOLVNDC.png
crypto\2024-12-08_02-40-24_XLMVNDC.png NOT EXITS !
crypto\2024-12-08_02-40-24_XLMVNDC.png
crypto\2024-12-08_02-41-16_LISTAVNDC.png NOT EXITS !
crypto\2024-12-08_02-41-16_LISTAVNDC.png
crypto\2024-12-08_02-46-53_HBARVNDC.png NOT EXITS !
crypto\2024-12-08_02-46-53_HBAR